In [2]:
import datetime, requests, time
import xml.etree.ElementTree as ET
import pandas as pd

In [3]:
# api calls
apiBase = 'https://www.boardgamegeek.com/xmlapi2/'
gameInfo = 'thing?id='
userInfo = 'user?name='
playsInfoByUser = 'plays?username='
playsInfoByGame = 'plays?id='

In [4]:
# retrieve API call
def get_game_metadata(gameIdList):
    """supply a list of game id's you want to retrive game metadata on.  Output is XML."""
    stringify = ','.join(gameIdList)
    response = requests.get(f'{apiBase}{gameInfo}{stringify}').content
    # add a short sleep so API doesn't throttle - we aren't making that makny calls
    time.sleep(5)
    return response

In [5]:
def listify_game_metadata(apiResponse):
    """The parses the game metadata XML into a list of lists for import"""
    gameMetadata = []
    gameCategories = []
    gameMechanisms = []
    importDate = datetime.datetime.strftime(datetime.datetime.now().date(), '%Y-%m-%d')

    # game info
    tree = ET.fromstring(apiResponse)

    # iterate through response top level 'item'
    for item in tree.findall('item'):

        # define metadata we want to collect since Type & Id are attributes at the top level we will set them here
        itemType = item.attrib['type']
        itemId = item.attrib['id']      
        itemName = None                 
        itemYearPublished = None                
        itemMinPlayers = None           
        itemMaxPlayers = None           
        itemPlayTime = None             
        itemCategories = []
        itemMechanisms = []
        itemThumbnail = None
        
        # get itemName
        names = item.findall('name')
        for name in names:
            if name.attrib['type'].lower() == 'primary':
                name = name.attrib['value']
                itemName = name
        
        # get itemYearPublished
        publishedYear = item.find('yearpublished')
        itemYearPublished = publishedYear.attrib['value']

        # get itemMinPlayers
        minPlayers = item.find('minplayers')
        itemMinPlayers = minPlayers.attrib['value']

        # get itemMaxPlayers
        maxPlayers = item.find('maxplayers')
        itemMaxPlayers = maxPlayers.attrib['value']

        # get itemPlayingTime
        playingTime = item.find('playingtime')
        itemPlayTime = playingTime.attrib['value']

        # get itemCategories
        categories = item.findall('link')
        for category in categories:
            if 'categor' in category.attrib['type'].lower():
                itemCategories.append(category.attrib['value'])

        # get itemCategories
        mechanisms = item.findall('link')
        for mechanism in mechanisms:
            if 'mechanic' in mechanism.attrib['type'].lower():
                itemMechanisms.append(mechanism.attrib['value'])

        # get itemThumbnail
        thumbnail = item.find('thumbnail')
        itemThumbnail = thumbnail.text

        gameMetadata += tuple([(itemId, itemType, itemName, itemYearPublished, None, itemMinPlayers, itemMaxPlayers,
                         itemPlayTime, itemThumbnail, importDate)])
        
        gameCategories += tuple([(itemId, x) for x in itemCategories])

        gameMechanisms += tuple([(itemId, x) for x in itemMechanisms])
        
    #return [gameMetadata]
    return [gameCategories]
    #return [gameMechanisms]

In [6]:

idList= ['174430', '161936'] #'224517']
categorylist=[]

for a in idList:
    response = get_game_metadata(str(a))
    res=listify_game_metadata(response)
    count=0
    for i in res:
        categorylist.append([a,res[count][1]])
        count+=1
        
    

    
categorylist



[['174430', ('1', 'Negotiation')], ['161936', ('1', 'Negotiation')]]

In [8]:
id_df=pd.read_csv('game_ranking.csv')
id_df.head()

,rank,id,name,description,bggrating,avgrating,numvoter,date
0,1,174430,Gloomhaven,Vanquish monsters with strategic cardplay. Ful...,8.534,8.77,44979,2021-08-10
1,2,161936,Pandemic Legacy: Season 1,Mutating diseases are spreading around the wor...,8.453,8.61,43369,2021-08-10
2,3,224517,Brass: Birmingham,"Build networks, grow industries, and navigate ...",8.403,8.67,22391,2021-08-10
3,4,167791,Terraforming Mars,Compete with rival CEOs to make Mars habitable...,8.279,8.43,69407,2021-08-10
4,5,291457,Gloomhaven: Jaws of the Lion,Vanquish monsters with strategic cardplay in a...,8.256,8.77,12192,2021-08-10


In [10]:
idlist=id_df['id'].tolist()
idlist

[174430,
 161936,
 224517,
 167791,
 291457,
 233078,
 220308,
 187645,
 182028,
 193738,
 162886,
 12333,
 115746,
 169786,
 84876,
 173346,
 120677,
 124361,
 28720,
 266192,
 167355,
 177736,
 205637,
 183394,
 164928,
 96848,
 237182,
 199792,
 3076,
 102794,
 175914,
 31260,
 170216,
 205059,
 221107,
 276025,
 316554,
 285774,
 180263,
 284083,
 192135,
 266507,
 247763,
 55690,
 126163,
 2651,
 185343,
 209010,
 164153,
 216132,
 251247,
 35677,
 230802,
 184267,
 72125,
 312484,
 25613,
 125153,
 256960,
 124742,
 191189,
 161533,
 121921,
 159675,
 521,
 244521,
 171623,
 28143,
 266810,
 229853,
 246900,
 201808,
 68448,
 157354,
 62219,
 110327,
 200680,
 122515,
 264220,
 236457,
 18602,
 93,
 182874,
 12493,
 37111,
 146021,
 73439,
 172386,
 40834,
 205896,
 269385,
 163412,
 170042,
 144733,
 42,
 203993,
 281259,
 178900,
 102680,
 132531,
 225694,
 36218,
 2511,
 30549,
 155821,
 198928,
 172287,
 233371,
 196340,
 127023,
 175155,
 266524,
 175640,
 263918,
 161970,
 

In [12]:
idList= ['174430', '161936']
categorylist=[]
for a in idlist:
    response = get_game_metadata(str(a))
    res=listify_game_metadata(response)
    for t in res[0]:
        categorylist.append([a,t[1]])
print(res)
categorylist

[[('2', 'Card Game'), ('2', 'Fantasy'), ('5', 'Economic'), ('5', 'Territory Building'), ('7', 'Abstract Strategy'), ('5', 'Economic'), ('5', 'Territory Building'), ('1', 'Economic'), ('1', 'Negotiation'), ('1', 'Political'), ('8', 'Civilization'), ('8', 'Fantasy')]]


[[174430, 'Economic'],
 [174430, 'Negotiation'],
 [174430, 'Political'],
 [174430, 'Abstract Strategy'],
 [174430, 'Ancient'],
 [174430, 'Ancient'],
 [174430, 'Abstract Strategy'],
 [174430, 'Medieval'],
 [161936, 'Economic'],
 [161936, 'Negotiation'],
 [161936, 'Political'],
 [161936, 'Civilization'],
 [161936, 'Nautical'],
 [161936, 'Economic'],
 [161936, 'Negotiation'],
 [161936, 'Political'],
 [161936, 'Exploration'],
 [161936, 'Abstract Strategy'],
 [161936, 'Medieval'],
 [161936, 'Civilization'],
 [161936, 'Nautical'],
 [224517, 'Card Game'],
 [224517, 'Fantasy'],
 [224517, 'Card Game'],
 [224517, 'Fantasy'],
 [224517, 'Ancient'],
 [224517, 'Economic'],
 [224517, 'Territory Building'],
 [224517, 'Economic'],
 [224517, 'Negotiation'],
 [224517, 'Political'],
 [224517, 'Abstract Strategy'],
 [167791, 'Economic'],
 [167791, 'Negotiation'],
 [167791, 'Political'],
 [167791, 'Civilization'],
 [167791, 'Nautical'],
 [167791, 'Abstract Strategy'],
 [167791, 'Abstract Strategy'],
 [16779

In [13]:
type(categorylist)

list

In [17]:
categories=pd.DataFrame(categorylist,columns=['id','categories'])
categories

,id,categories
0,174430,Economic
1,174430,Negotiation
2,174430,Political
3,174430,Abstract Strategy
4,174430,Ancient
...,...,...
9455,257518,Economic
9456,257518,Negotiation
9457,257518,Political
9458,257518,Civilization


In [18]:
categories.to_csv('bgg_categories.csv')

In [22]:
categories['categories'] = categories.groupby(['id'])['categories'].transform(lambda x : ','.join(x))

In [25]:
categories = categories.drop_duplicates()
categories

,id,categories
0,174430,"Economic,Negotiation,Political,Abstract Strate..."
8,161936,"Economic,Negotiation,Political,Civilization,Na..."
21,224517,"Card Game,Fantasy,Card Game,Fantasy,Ancient,Ec..."
32,167791,"Economic,Negotiation,Political,Civilization,Na..."
43,291457,"Card Game,Fantasy,Exploration,Economic,Negotia..."
...,...,...
9409,204836,"Card Game,Fantasy,Ancient,Civilization,Fantasy..."
9418,141423,"Economic,Negotiation,Political,Ancient,Economi..."
9430,244049,"Card Game,Fantasy,Ancient,Ancient,Ancient,Expl..."
9436,284818,"Card Game,Fantasy,Civilization,Fantasy,Ancient..."


In [26]:
categories.to_csv('bgg_categories_drop.csv')

In [27]:
game_ranking=pd.read_csv('game_ranking.csv')
game_ranking.head()

,rank,id,name,description,bggrating,avgrating,numvoter,date
0,1,174430,Gloomhaven,Vanquish monsters with strategic cardplay. Ful...,8.534,8.77,44979,2021-08-10
1,2,161936,Pandemic Legacy: Season 1,Mutating diseases are spreading around the wor...,8.453,8.61,43369,2021-08-10
2,3,224517,Brass: Birmingham,"Build networks, grow industries, and navigate ...",8.403,8.67,22391,2021-08-10
3,4,167791,Terraforming Mars,Compete with rival CEOs to make Mars habitable...,8.279,8.43,69407,2021-08-10
4,5,291457,Gloomhaven: Jaws of the Lion,Vanquish monsters with strategic cardplay in a...,8.256,8.77,12192,2021-08-10


In [29]:
ranking_category = pd.merge(game_ranking,categories, on ='id',how ='left')
ranking_category

,rank,id,name,description,bggrating,avgrating,numvoter,date,categories
0,1,174430,Gloomhaven,Vanquish monsters with strategic cardplay. Ful...,8.534,8.77,44979,2021-08-10,"Economic,Negotiation,Political,Abstract Strate..."
1,2,161936,Pandemic Legacy: Season 1,Mutating diseases are spreading around the wor...,8.453,8.61,43369,2021-08-10,"Economic,Negotiation,Political,Civilization,Na..."
2,3,224517,Brass: Birmingham,"Build networks, grow industries, and navigate ...",8.403,8.67,22391,2021-08-10,"Card Game,Fantasy,Card Game,Fantasy,Ancient,Ec..."
3,4,167791,Terraforming Mars,Compete with rival CEOs to make Mars habitable...,8.279,8.43,69407,2021-08-10,"Economic,Negotiation,Political,Civilization,Na..."
4,5,291457,Gloomhaven: Jaws of the Lion,Vanquish monsters with strategic cardplay in a...,8.256,8.77,12192,2021-08-10,"Card Game,Fantasy,Exploration,Economic,Negotia..."
...,...,...,...,...,...,...,...,...,...
995,996,204836,Escape Room: The Game,Solve puzzles and use the keys to stop the chr...,6.528,7.04,3418,2021-08-10,"Card Game,Fantasy,Ancient,Civilization,Fantasy..."
996,997,141423,Dead Men Tell No Tales,Work together as pirates to defeat enemies and...,6.527,7.13,3091,2021-08-10,"Economic,Negotiation,Political,Ancient,Economi..."
997,998,244049,Forum Trajanum,Find out who's hiding in your Roman settlement...,6.527,7.32,2172,2021-08-10,"Card Game,Fantasy,Ancient,Ancient,Ancient,Expl..."
998,999,284818,Caylus 1303,Quarrel over action spaces (and player powers)...,6.525,7.60,1579,2021-08-10,"Card Game,Fantasy,Civilization,Fantasy,Ancient..."


In [30]:
ranking_category.to_csv('ranking_category.csv')